In [1]:
from pathlib import Path
from PIL import Image
from PyPDF2 import PdfMerger

In [ ]:
DOWNLOAD_FOLDER = Path("data")
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tiff", ".gif"}

ROOT_FOLDER = Path.cwd()
output_pdf_folder = ROOT_FOLDER / "pdf"
output_pdf_folder.mkdir(exist_ok=True)

In [ ]:
for manga_dir in DOWNLOAD_FOLDER.iterdir():
    if not manga_dir.is_dir():
        continue

    print(f"\n=== Обробка манги: {manga_dir.name} ===")

    full_pdf_allowed = True
    chapter_pdfs = []

    for chapter_dir in sorted(manga_dir.iterdir(), key=lambda x: x.name):
        if not chapter_dir.is_dir():
            continue

        images_check = sorted(
            [
                img for img in chapter_dir.iterdir()
                if img.suffix.lower() in IMAGE_EXTS
            ],
            key=lambda x: x.name
        )

        images = [img for img in images_check if img.stat().st_size > 1024]

        if not images:
            print(f"  {chapter_dir.name} — немає картинок")
            full_pdf_allowed = False
            continue

        if len(images) != len(images_check):
            print(f"  {chapter_dir.name} — биті картинки, пропуск full.pdf")
            full_pdf_allowed = False

        pdf_path = manga_dir / f"{chapter_dir.name}.pdf"

        with Image.open(images[0]).convert("RGB") as first_img:
            rest_imgs = []

            for img_path in images[1:]:
                with Image.open(img_path).convert("RGB") as im:
                    rest_imgs.append(im.copy())

            first_img.save(
                pdf_path,
                save_all=True,
                append_images=rest_imgs
            )

        print(f"  PDF створено: {pdf_path}")
        chapter_pdfs.append(pdf_path)

    # ─── загальний PDF ──────────────────────────────────────────────
    if chapter_pdfs:
        total_pdf_path = manga_dir / f"{manga_dir.name}_full"
        if full_pdf_allowed:
            total_pdf_path = total_pdf_path.with_suffix(".pdf")
        else:
            total_pdf_path = (total_pdf_path.parent / (total_pdf_path.name + "_cracked")).with_suffix(".pdf")

        merger = PdfMerger()
        for pdf in chapter_pdfs:
            merger.append(str(pdf))

        merger.write(total_pdf_path)
        merger.close()

        print(f"\n=== Загальний PDF створено: {total_pdf_path} ===")


=== Обробка манги: Цього разу я володар свого життя ===
  PDF створено: data/Цього разу я володар свого життя/0001_26392-cogo-razu-ja-gospodarka-svogo-zhittja-tom-1-rozdil-0.pdf
  PDF створено: data/Цього разу я володар свого життя/0002_8008-cogo-razu-ja-gospodarka-svogo-zhittja-tom-1-rozdil-1.pdf
  PDF створено: data/Цього разу я володар свого життя/0003_8384-cogo-razu-ja-gospodarka-svogo-zhittja-tom-1-rozdil-2.pdf
  PDF створено: data/Цього разу я володар свого життя/0004_10425-cogo-razu-ja-gospodarka-svogo-zhittja-tom-1-rozdil-3.pdf
  PDF створено: data/Цього разу я володар свого життя/0005_10626-cogo-razu-ja-gospodarka-svogo-zhittja-tom-1-rozdil-4.pdf
  PDF створено: data/Цього разу я володар свого життя/0006_11030-cogo-razu-ja-gospodarka-svogo-zhittja-tom-1-rozdil-5.pdf
  PDF створено: data/Цього разу я володар свого життя/0007_11898-cogo-razu-ja-gospodarka-svogo-zhittja-tom-1-rozdil-6.pdf
  PDF створено: data/Цього разу я володар свого життя/0008_12522-cogo-razu-ja-gospodarka-sv

In [5]:
for manga_dir in DOWNLOAD_FOLDER.iterdir():
    if not manga_dir.is_dir():
        continue

    for pdf_file in manga_dir.glob("*_full.pdf"):
        pdf_file.rename(output_pdf_folder / pdf_file.name)

    for pdf_file in manga_dir.glob("*_full_cracked.pdf"):
        pdf_file.rename(output_pdf_folder / pdf_file.name)

print(f"\n=== Всі загальні PDF файли переміщено до папки: {output_pdf_folder} ===")


=== Всі загальні PDF файли переміщено до папки: /Users/aleksejkitajskij/Desktop/mangainua_parser/pdf ===
